In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import optimize

In [3]:
from BHDVCStf import BHDVCS

In [4]:
bhdvcs = BHDVCS()

In [5]:
data = pd.read_csv('dvcs_psuedo.csv')

In [1]:
import utilities as uts
from tqdm import tqdm

In [6]:
data = data.loc[(data['#Set'] == 0) & (data['phi_x'] <= 180) & (data['phi_x'] != 90), :]

In [8]:
def produceLHS(data, bhdvcs):
    bhdvcs.SetKinematics(data['QQ'], data['x_b'], data['t'], data['k'])
    bh = bhdvcs.GetBHUUxs(data['phi_x'], data['F1'], data['F2'])
    interference = data['F'] - data['dvcs'] - bh
    interference /= bhdvcs.Gamma
    interference *= data['QQ']
    interference *= np.abs(data['t'])
    a, b = bhdvcs.ABcon(data['phi_x'], data['F1'], data['F2'])
    interference /= b
    return interference

In [9]:
def produceLHSerr(data, bhdvcs):
    bhdvcs.SetKinematics(data['QQ'], data['x_b'], data['t'], data['k'])
    err = data['errF']
    err /= bhdvcs.Gamma
    err *= data['QQ']
    err *= np.abs(data['t'])
    a, b = bhdvcs.ABcon(data['phi_x'], data['F1'], data['F2'])
    err /= b
    return np.abs(err)

In [10]:
def aoverb(data, bhdvcs):
    bhdvcs.SetKinematics(data['QQ'], data['x_b'], data['t'], data['k'])
    a, b = bhdvcs.ABcon(data['phi_x'], data['F1'], data['F2'])
    return a/b

In [37]:
def linear_curve_fit(data, ReH, ReE):
    bhdvcs.SetKinematics(data[:, 0], data[:, 1], data[:, 2], data[:, 3])
    a, b = bhdvcs.ABcon(data[:, 4], data[:, 5], data[:, 6])
    #tau = -data['t']/(4*bhdvcs.M2)
    gm = data[:, 5] + data[:, 6]
    return (a/b)*(data[:, 5]*ReH + bhdvcs.tau*data[:, 6]*ReE) + gm*(ReH + ReE)

In [33]:
def combined_function_chisq(cffs, data):
    ReH, ReE, ReHtilde = cffs
    dftu = data.copy()
    dftu['F'] = bhdvcs.TotalUUXS(np.array(data[['phi_x', 'k', 'QQ', 'x_b', 't', 'F1', 'F2', 'dvcs']]), ReH, ReE, ReHtilde)
    yhat1 = produceLHS(dftu, bhdvcs)
    X = np.array(data[['QQ', 'x_b', 't', 'k', 'phi_x', 'F1', 'F2']])
    yhat2 = linear_curve_fit(X, ReH, ReE)
    chisq1 = np.sum(((yhat1 - data['F'])/data['errF'])**2)
    chisq2 = np.sum(((yhat2 - data['F'])/data['errF'])**2)
    return (chisq1 + chisq2)/2

In [28]:
data = pd.read_csv('dvcs_psuedo.csv')

In [29]:
high = data.copy()
high['F'] = high['F'] + high['errF']
data['errF'] = np.abs(produceLHS(high, bhdvcs) - produceLHS(data, bhdvcs))

In [30]:
data['F'] = produceLHS(data, bhdvcs)

## Set 1

In [119]:
test_data = data.loc[(data['#Set'] == 0) & (data['phi_x'] <= 180) & (data['phi_x'] != 90), :]

In [120]:
x0 = np.array([1, 1, 1])
res = optimize.minimize(combined_function_chisq, x0, args=test_data)

In [121]:
res.x

array([ 15.8791021 , -68.81298446,   3.76043883])

In [122]:
np.diag(res.hess_inv)

array([ 1.12425836, 29.73266721,  0.38083102])

## Set 2

In [123]:
test_data = data.loc[(data['#Set'] == 1) & (data['phi_x'] <= 180) & (data['phi_x'] != 90), :]

In [124]:
x0 = np.array([1, 1, 1])
res = optimize.minimize(combined_function_chisq, x0, args=test_data)

In [125]:
res.x

array([  22.80075777, -106.27311581,    3.14101246])

In [126]:
np.diag(res.hess_inv)

array([ 2.65185418, 72.37910705,  1.18399464])

In [100]:
seti = data.loc[(data['#Set'] == i) & (data['phi_x'] <= 180) & (data['phi_x'] != 90), :]
rep_data = seti.copy()
rep_data['F'] = np.random.normal(rep_data['F'], rep_data['errF'])

In [101]:
rep_data

,#Set,index,errF,F,phi_x,QQ,x_b,t,k,F1,F2,dvcs,ReH,ReE,ReHtilde
72,2,0,7.386159,-36.449778,0,2.17955,0.385579,-0.283318,4.75,0.579414,0.848208,0.012288,7.22424,-47.2242,4.01347
73,2,1,7.308711,-48.273208,10,2.17955,0.385579,-0.283318,4.75,0.579414,0.848208,0.012288,7.22424,-47.2242,4.01347
74,2,2,7.293302,-23.867232,20,2.17955,0.385579,-0.283318,4.75,0.579414,0.848208,0.012288,7.22424,-47.2242,4.01347
75,2,3,7.917898,-24.454727,30,2.17955,0.385579,-0.283318,4.75,0.579414,0.848208,0.012288,7.22424,-47.2242,4.01347
76,2,4,8.741887,-36.001161,40,2.17955,0.385579,-0.283318,4.75,0.579414,0.848208,0.012288,7.22424,-47.2242,4.01347
77,2,5,9.778452,-36.725486,50,2.17955,0.385579,-0.283318,4.75,0.579414,0.848208,0.012288,7.22424,-47.2242,4.01347
78,2,6,11.226435,-31.586213,60,2.17955,0.385579,-0.283318,4.75,0.579414,0.848208,0.012288,7.22424,-47.2242,4.01347
79,2,7,16.054382,-30.243423,70,2.17955,0.385579,-0.283318,4.75,0.579414,0.848208,0.012288,7.22424,-47.2242,4.01347
80,2,8,31.530305,-78.905321,80,2.17955,0.385579,-0.283318,4.75,0.579414,0.848208,0.012288,7.22424,-47.2242,4.01347
82,2,10,26.400642,-78.402304,100,2.17955,0.385579,-0.283318,4.75,0.579414,0.848208,0.012288,7.22424,-47.2242,4.01347


In [108]:
np.array([, np.random.normal(res.x, np.diag(res.hess_inv), size=(10, 3))])

array([[[  19.89816508,  -88.34121367,    4.16433741],
        [  25.51979272, -115.75732595,    4.59676969],
        [  23.56839302,  -72.88467036,    2.53200944],
        [  26.86312547,  -91.74153215,    3.46497155],
        [  20.73871708, -287.54265193,    3.57678543],
        [  24.01563157,    1.20282739,    3.06943454],
        [  21.69141536, -140.09653075,    4.24545324],
        [  22.65614068,  -77.97815981,    3.91353327],
        [  22.59310269,  -99.92002087,    2.71104807],
        [  18.50843565, -197.46597487,    4.00077812]],

       [[  20.62384541, -160.63609382,    1.19334125],
        [  26.99151647,   23.83793832,    3.50546693],
        [  23.67283548, -115.75091353,    3.94521585],
        [  19.53855812, -152.97019707,    2.45278248],
        [  23.23834906,   34.35692924,    3.90016106],
        [  21.72492143, -142.7226738 ,    3.89953589],
        [  24.27088822,   -3.68752385,    0.76589982],
        [  24.34931396, -256.00170037,    0.90076853],
        

In [136]:
def produceResults(data, numSets, numReplicas):
    '''
    :param data: data of class DvcsData (which can be found in utilities.py)
    :param numSets: number of different kinematic sets contained in data
    :param numReplicas: number of replicas you would like to produce
    
    :returns: numpy array with shape (numSets, numReplicas, numCFFs)
    '''
    results = []
    for i in tqdm(range(numSets)):
        seti = data.loc[(data['#Set'] == i) & (data['phi_x'] <= 180) & (data['phi_x'] != 90), :]
        replicas = []
        #X = np.array(seti.XnoCFF) # the kinematics and all variables necessary to compute 
        #sigma = seti.erry 
        pars = np.array([1, 1, 1])
        optobj = optimize.minimize(combined_function_chisq, x0, args=seti)
        results.append(np.random.normal(res.x, np.diag(res.hess_inv), size=(numReplicas, 3)))
    return np.array(results)

In [110]:
results = produceResults(data, 15, 10)

100%|██████████| 15/15 [02:24<00:00,  9.61s/it]


In [115]:
localFits = np.fromfile('replicas500.txt')
localFits = localFits.reshape(15, 500, 3)